In [ ]:
import cv2
import os
import random
import numpy as np
import time
import uuid
from matplotlib import pyplot as plt
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
import albumentations as alb
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model


In [ ]:
img = cv2.imread(os.path.join('data','test', 'images','b8785bd4-ff27-11ee-bfc0-ae6d906d4950.jpg'))

In [ ]:
with open(os.path.join('data', 'test', 'labels', 'b8785bd4-ff27-11ee-bfc0-ae6d906d4950.json'), 'r') as f:
    label = json.load(f)

In [ ]:
import cv2
import albumentations as A
import matplotlib.pyplot as plt

# Extract the bounding box coordinates
x_min, y_min = label['shapes'][0]['points'][0]
x_max, y_max = label['shapes'][0]['points'][1]

# Get the image dimensions
height, width = img.shape[:2]

# Normalize the bounding box coordinates to the range [0.0, 1.0]
bbox = [x_min / width, y_min / height, x_max / width, y_max / height]

transform = A.Compose([
    A.RandomCrop(width=1000, height=1000),
    alb.HorizontalFlip(p=0.5), 
    alb.RandomBrightnessContrast(p=0.2),
    alb.RandomGamma(p=0.2), 
    alb.RGBShift(p=0.2), 
    alb.VerticalFlip(p=0.5)
], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels']))

transformed = transform(image=img, bboxes=[bbox], class_labels=['face'])

cropped_img = transformed['image']
cropped_labels = transformed['bboxes']

# Convert the cropped image from BGR to RGB color space
cropped_img_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)

# Extract the cropped bounding box coordinates
if cropped_labels:
    x_min, y_min, x_max, y_max = cropped_labels[0]

    # Convert the normalized coordinates back to pixel values
    x_min = int(x_min * 1000)
    y_min = int(y_min * 1000)
    x_max = int(x_max * 1000)
    y_max = int(y_max * 1000)

    # Draw the cropped bounding box on the cropped image
    cv2.rectangle(cropped_img_rgb, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

# Display the cropped image with the bounding box using Matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(cropped_img_rgb)
plt.axis('off')
plt.title('Cropped Image with Bounding Box')
plt.tight_layout()
plt.show()


In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            # Extract the bounding box coordinates
            x_min, y_min = label['shapes'][0]['points'][0]
            x_max, y_max = label['shapes'][0]['points'][1]

            # Get the image dimensions
            height, width = img.shape[:2]

            # Normalize the bounding box coordinates to the range [0.0, 1.0]
            bbox = [x_min / width, y_min / height, x_max / width, y_max / height]

        try: 
            for x in range(80):
                augmented = transform(image=img, bboxes=[bbox], class_labels=['face'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_physical_devices('GPU')

In [ ]:
def load_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image


In [ ]:
train_images = tf.data.Dataset.list_files('aug_data/train/images/*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120, 120)))
train_images = train_images.map(lambda x: x / 255)


In [ ]:
train_images

In [ ]:
test_images = tf.data.Dataset.list_files('aug_data/test/images/*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x,(120,120)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
test_images

In [ ]:
val_images = tf.data.Dataset.list_files('aug_data/val/images/*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x,(120,120)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
val_images

In [ ]:
import tensorflow as tf
import json

def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding="utf-8") as f:
        label = json.load(f)
    return [label['class']], label['bbox']

def load_labels_wrapper(label_path):
    output = tf.py_function(load_labels, [label_path], [tf.uint8, tf.float16])
    # Directly set the shapes of the tensors without unpacking
    output[0].set_shape([1])  # Setting the shape for class label tensor
    output[1].set_shape([4])  # Assuming bbox contains 4 elements [x_min, y_min, x_max, y_max]
    return output


In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data/train/labels/*.json', shuffle=False)
train_labels = train_labels.map(load_labels_wrapper)

train_labels


In [ ]:
test_labels = tf.data.Dataset.list_files('aug_data/test/labels/*.json', shuffle=False)
test_labels = test_labels.map(load_labels_wrapper)

test_labels


In [ ]:
val_labels = tf.data.Dataset.list_files('aug_data/val/labels/*.json', shuffle=False)
val_labels = val_labels.map(load_labels_wrapper)

val_labels


In [ ]:
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(8)
train = train.prefetch(4)

In [ ]:
train

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
test

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
val

In [ ]:
train.as_numpy_iterator().next()[1]

In [ ]:
data_samples = train.as_numpy_iterator()
res = data_samples.next()

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]
    
    # Create a copy of the sample_image
    sample_image_copy = np.copy(sample_image)
    
    cv2.rectangle(sample_image_copy, 
                  tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                        (255,0,0), 2)

    ax[idx].imshow(sample_image_copy)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
vgg = VGG16(include_top=False)

In [ ]:
def build_model(input_shape=(120, 120, 3)):
    input_layer = Input(shape=input_shape)
    base_model = VGG16(include_top=False, input_tensor=input_layer)
    
    # Feature extraction
    pooled_features = GlobalMaxPooling2D()(base_model.output)
    
    # Classification head
    classifier_output = Dense(2048, activation='relu')(pooled_features)
    classifier_output = Dense(1, activation='sigmoid', name='class_output')(classifier_output)
    
    # Regression head
    regressor_output = Dense(2048, activation='relu')(pooled_features)
    regressor_output = Dense(4, activation='sigmoid', name='bbox_output')(regressor_output)
    
    # Full model
    model = Model(inputs=input_layer, outputs=[classifier_output, regressor_output])
    return model


In [ ]:
def localization_loss(y_true, yhat):            
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))
                  
    h_true = y_true[:,3] - y_true[:,1] 
    w_true = y_true[:,2] - y_true[:,0] 

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord + delta_size

In [ ]:
# Build and compile the model
facetracker = build_model()
facetracker.summary()
batches_per_epoch = len(train)
lr_decay = (1. / 0.75 - 1) / batches_per_epoch
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=lr_decay)

facetracker.compile(optimizer=optimizer,
                    loss={'class_output': tf.keras.losses.BinaryCrossentropy(),
                          'bbox_output': localization_loss},
                    metrics={'class_output': ['accuracy'], 'bbox_output': []})

# Callbacks and training
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')
hist = facetracker.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])


In [ ]:
import matplotlib.pyplot as plt

# Extracting data from the training history
history_dict = hist.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['class_output_accuracy']  # This might differ based on your specific outputs
val_acc_values = history_dict['val_class_output_accuracy']  # This might differ too

epochs = range(1, len(loss_values) + 1)

# Plotting the training and validation loss
plt.figure(figsize=(12,6))

plt.subplot(1, 2, 1)
plt.plot(epochs, loss_values, label='Training loss')
plt.plot(epochs, val_loss_values, label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, acc_values, label='Training accuracy')
plt.plot(epochs, val_acc_values, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
test_sample = test_data.next()

In [ ]:
yhat = facetracker.predict(test_sample[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx].copy()  # Create a copy of the image array
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120,120]).astype(int)), 
                            (255,0,0), 2)
    
    ax[idx].imshow(sample_image)


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
facetracker.save('facetracker.h5')

In [ ]:
facetracker = load_model('facetracker.h5')

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [frame.shape[1],frame.shape[0]]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [frame.shape[1],frame.shape[0]]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [frame.shape[1],frame.shape[0]]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [frame.shape[1],frame.shape[0]]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [frame.shape[1],frame.shape[0]]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
